In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import re, string

In [7]:
data=pd.read_csv(\
                     'https://s3.amazonaws.com/danicic-w210/combined_trainingdata_20181013.tsv',sep='\t')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#filePath = "../Datasets/combined_trainingdata_20181013.tsv"
#data = pandas.read_csv(filePath,sep='\t')
print(data.shape)
print(data.loc[0])

print(data.columns[data.isna().any()].tolist())
print(data[data.COMPLAINT_1.isna()].shape)
print(data[data.COMPLAINT_2.isna()].shape)
print(data[(data.COMPLAINT_1.isna()) & (data.COMPLAINT_2.isna())].shape)

dataFiltered = data.dropna(subset = ["COMPLAINT_1"])
print(dataFiltered[(dataFiltered.COMPLAINT_1.isna())].shape)
print(dataFiltered[(dataFiltered.COMPLAINT_1.isna()) & (dataFiltered.COMPLAINT_2.isna())].shape)
print(dataFiltered[dataFiltered.COMPLAINT_1 == ""].shape)

(1278129, 10)
index                                                             0
COMPLAINT_ID                                        US_CHICAGO_1725
CITY                                                     US_CHICAGO
COMPLAINT DATE                                           10/03/2011
DEPT_311                                         health_environment
CODE_311                           permits issued by doe work order
CATEGORY_MAIN                                           environment
CATEGORY_SUB                                    environment_general
COMPLAINT_1       QUESTIONABLE BUSINESS PRACTICES REGARDING OILS...
COMPLAINT_2       [INSPECTION LOG #: 1723 03-OCT-11 18:55:00] TH...
Name: 0, dtype: object
['COMPLAINT DATE', 'DEPT_311', 'COMPLAINT_1', 'COMPLAINT_2']
(14025, 10)
(964291, 10)
(13579, 10)
(0, 10)
(0, 10)
(0, 10)


In [5]:
translator = str.maketrans('', '', string.punctuation) # To remove punctuation

def preProcess(complaintStart):
    complaint = complaintStart[:512] # cut to 512 characters max
    complaint = re.sub("\d","N", complaint) # remove numbers
    complaint = complaint.lower().translate(translator) # lower case and remove the punctuation
    complaint = complaint.replace("\n"," ").strip() # remove starting and trailing white spaces
    if re.search('[a-zA-Z]', complaint) is None:# if there are no letters in the complaint, return empty, will be removed in later processing
        return ""
    return complaint

def getComplaint(row):
    complaint2 = row.get("COMPLAINT_2")
    if not pd.isnull(complaint2):
        if "[INSPECTION LOG #:" in complaint2: # Remove inspection log section from C2
            complaintStrippedList = complaint2.split("]")[1:]
            complaintFinal = "]".join(complaintStrippedList)
        else:
            complaintFinal = complaint2
        if row.get("CITY")=="US_CHICAGO": # if Chicago, concatenate the two
            complaintFinal = row.get("COMPLAINT_1") + " "+ complaintFinal
        complaintProcessed = preProcess(complaintFinal)
        if complaintProcessed == "" or re.search('[a-zA-Z]', complaintProcessed) is None: # if nothing or no letters
            return preProcess(row.get("COMPLAINT_1"))
        return complaintProcessed
    complaintProcessed = preProcess(row.get("COMPLAINT_1"))
    return complaintProcessed

results = dataFiltered.apply(lambda row: getComplaint (row),axis=1)
print(results[results.isna()].shape)

dataFiltered["complaint"] = results
print(dataFiltered[dataFiltered.complaint.isna()].shape)

dataFiltered["CATEGORY_SUB"] = dataFiltered["CATEGORY_SUB"].str.strip()

(0,)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(0, 11)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Start Cleaning

**Remove any comment that is less than 2 characters long**

In [6]:
mask = (dataFiltered["complaint"].str.len() >=2)
dataFiltered = dataFiltered.loc[mask]
print(dataFiltered.shape)

(1159564, 11)


**lets look at everything that occurs 5 times or more**

In [ ]:
df5plus=pd.DataFrame(dataFiltered["complaint"].value_counts()[dataFiltered["complaint"].value_counts()>=5])

In [20]:
dataFiltered[dataFiltered["complaint"].str.len() ==4].groupby('complaint').count().reset_index().sort_values('index', ascending=False).head(20)

,complaint,index,COMPLAINT_ID,CITY,COMPLAINT DATE,DEPT_311,CODE_311,CATEGORY_MAIN,CATEGORY_SUB,COMPLAINT_1,COMPLAINT_2
107,mold,696,696,696,696,691,696,696,696,696,5
103,mice,513,513,513,513,497,513,513,513,513,16
150,rats,215,215,215,215,213,215,215,215,215,1
175,sofa,180,180,180,180,169,180,180,180,180,11
147,pous,151,151,151,151,151,151,151,151,151,0
186,tire,126,126,126,126,119,126,126,126,126,6
184,test,107,107,107,107,38,107,107,107,107,68
53,desk,88,88,88,88,87,88,88,88,88,1
57,door,82,82,82,82,81,82,82,82,82,1
135,pcgs,73,73,73,73,73,73,73,73,73,0


** Let's look at some example potential autofills**

In [9]:
dataFiltered[dataFiltered["complaint"]=='waze user reported'].groupby(["CATEGORY_MAIN","CATEGORY_SUB"]).count()

,,index,COMPLAINT_ID,CITY,COMPLAINT DATE,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,complaint
CATEGORY_MAIN,CATEGORY_SUB,,,,,,,,,
street,street_repair,267,267,267,267,267,267,267,0,267


In [25]:
any(substring in 'reported issue online' for substring in ['hola','report'])

True

In [28]:
df5plus[df5plus.index.map(lambda x: any(substring in x for substring in ['system','online','report']))]

,complaint
this issue was reported to the city of oakland public works department via phone nnnnnnnnnn email pwacallcenteroaklandnetcom or web wwwoaklandpwcom,5027
this issue was reported to the city of oakland call center via phone nnnnnnnnnn email callcenteroaklandnetcom or web wwwoaklandpwcom,4613
this issue was reported to the city of oakland public works agency via phone nnnnnnnnnn email pwacallcenteroaklandnetcom or web wwwoaklandpwcom,4472
this issue was reported to oak nnn via phone nnn or nnnnnnnnnn from outside oakland email oaknnnoaklandnetcom or web nnnoaklandcagov,1118
waze user reported,267
this issue was reported to the city of ann arbor customer service via phone nnn nnnnnnn email customerserviceangovorg or web wwwangovorg,207
reported by nn,156
need vehicle tow report,118
report a litter hotspot,79
resident originally called in on nnnnn and put in request nnnnnn for new resident recycling cart due to the change in contractors we are updating system with new request per nelle fruge sanitation manager,64


** Auto text to remove **

In [29]:
# strip these

strip_list=['transfer nnnnnn nnn',\
        'request entered through the web refer to intake questions for further description',\
       'more information may be available in the cdph environmental inspections dataset',\
       '',\
       '']

,complaint
request entered through the web refer to intake questions for further description,141284
graffiti removal,17243
pothole in street,16783
duplicate,16312
street light n out,11916


this issue was reported to the city of oakland public works agency via phone nnnnnnnnnn email 
pwacallcenteroaklandnetcom or web wwwoaklandpwcom

bu

rtc

mt airy nep nnnn

batch group nn

this issue was reported to oak nnn via phone nnn or nnnnnnnnnn from outside oakland email 
oaknnnoaklandnetcom or web nnnoaklandcagov

waze user reported

this issue was reported to the city of ann arbor customer service via phone nnn nnnnnnn email customerserviceangovorg or web wwwangovorg

reported by nn

council member smitherman reporting

nnn reported ps nnn down

test report

p c g s b

junkyard issues more information may be available in the cdph environmental inspections dataset

reissue ccnnnnnnnn to update

In [ ]:
# eliminate entire row of these complaints

elim_list=['this issue was reported to the city of oakland public works agency via phone nnnnnnnnnn email pwacallcenteroaklandnetcom or web wwwoaklandpwcom',\
          'bu',\
          'rtc',\
          'mt airy nep nnnn',\
          'batch group nn',\
          'this issue was reported to oak nnn via phone nnn or nnnnnnnnnn from outside oakland email oaknnnoaklandnetcom or web nnnoaklandcagov',\
          'waze user reported',\
          'this issue was reported to the city of ann arbor customer service via phone nnn nnnnnnn email customerserviceangovorg or web wwwangovorg',\
          'reported by nn',\
          'council member smitherman reporting',\
          'nnn reported ps nnn down',\
          'test report',\
          'p c g s b',\
          'junkyard issues more information may be available in the cdph environmental inspections dataset',\
          'reissue ccnnnnnnnn to update']

# Merge with Alex's

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import re, string

from pprint import pprint
import time
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
data=pd.read_csv(\
                     'https://s3.amazonaws.com/danicic-w210/combined_trainingdata_20181013.tsv',sep='\t')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
#filePath = "../Datasets/combined_trainingdata_20181013.tsv"
#data = pandas.read_csv(filePath,sep='\t')
print(data.shape)
print(data.loc[0])

print(data.columns[data.isna().any()].tolist())
print(data[data.COMPLAINT_1.isna()].shape)
print(data[data.COMPLAINT_2.isna()].shape)
print(data[(data.COMPLAINT_1.isna()) & (data.COMPLAINT_2.isna())].shape)

dataFiltered = data.dropna(subset = ["COMPLAINT_1"])
print(dataFiltered[(dataFiltered.COMPLAINT_1.isna())].shape)
print(dataFiltered[(dataFiltered.COMPLAINT_1.isna()) & (dataFiltered.COMPLAINT_2.isna())].shape)
print(dataFiltered[dataFiltered.COMPLAINT_1 == ""].shape)

(1278129, 10)
index                                                             0
COMPLAINT_ID                                        US_CHICAGO_1725
CITY                                                     US_CHICAGO
COMPLAINT DATE                                           10/03/2011
DEPT_311                                         health_environment
CODE_311                           permits issued by doe work order
CATEGORY_MAIN                                           environment
CATEGORY_SUB                                    environment_general
COMPLAINT_1       QUESTIONABLE BUSINESS PRACTICES REGARDING OILS...
COMPLAINT_2       [INSPECTION LOG #: 1723 03-OCT-11 18:55:00] TH...
Name: 0, dtype: object
['COMPLAINT DATE', 'DEPT_311', 'COMPLAINT_1', 'COMPLAINT_2']
(14025, 10)
(964291, 10)
(13579, 10)
(0, 10)
(0, 10)
(0, 10)


In [ ]:
# Alex's
def clean_specifics(complaint):
    complaint = re.sub('Request entered through the Web. Refer to Intake Questions for further description.',
                      '', complaint)
    complaint = re.sub('Transfer:.+/[A-Z]+', '', complaint)
    complaint = re.sub('ACCT ', '', complaint)
    complaint = re.sub('RTC ', '', complaint)
    return complaint
  
  
 # Preprocess `merged_complaint`
cachedStopWords = stopwords.words("english")

#def preprocess(complaint):
    #complaint = clean_specifics(complaint)
    #complaint = re.sub('\d', '', complaint) # Remove numbers completely
    #complaint = re.sub('[^\w\s]', ' ', complaint) # Sub puncuation with space
    #complaint = complaint.strip()
    #complaint = re.sub(' +', ' ', complaint) # Remove dupe spaces
    #complaint = complaint.lower()
    #complaint = ' '.join([word for word in complaint.split() if word not in cachedStopWords])
#    return complaint

In [ ]:
# eliminate stop phrases
# remove rows where complaint len <4

stop_phrases=['request entered web refer intake questions description',\
             'duplicate',\
             'batch created',\
             'issue reported city oakland public works department via phone email pwacallcenteroaklandnetcom web wwwoaklandpwcom',\
             'issue reported city oakland call center via phone email callcenteroaklandnetcom web wwwoaklandpwcom',\
             'issue reported city oakland public works agency via phone email pwacallcenteroaklandnetcom web wwwoaklandpwcom',\
             'issue reported oak via phone outside oakland email oakoaklandnetcom web oaklandcagov',\
             'request entered web refer intake questions description',\
             'waze user reported',\
             'council member smitherman reporting',\
             'issue reported city ann arbor customer service via phone email customerserviceagovorg web wwwagovorg',\
             'test report',\
             'reported mobile device httpmseeclickfixcom',\
             'information may available cdph environmental inspections dataset',\
             'ftc',\
             'resident reports',\
             'reported'\
             'rtc',\
             'acct',\
             'batch group'\
             'mt airy nep']

In [15]:
# Stan's + Alex

translator = str.maketrans('', '', string.punctuation) # To remove punctuation

def preProcess(complaintStart):
    complaint = clean_specifics(complaintStart)
    complaint = ' '.join([word for word in complaint.split() if word not in cachedStopWords]) # remove stopwords (alex) early on to make 512 limit 
    complaint = complaintStart[:512] # cut to 512 characters max
    complaint = re.sub("\d","", complaint) # remove numbers completely (alex)
    complaint = complaint.lower().translate(translator) # lower case and remove the punctuation
    complaint = re.sub('[^\w\s]', ' ', complaint) # Sub puncuation with space (alex)
    complaint = complaint.strip() # (alex)
    complaint = re.sub(' +', ' ', complaint) # Remove dupe spaces (alex)
    complaint = complaint.replace("\n"," ").strip() # remove starting and trailing white spaces
    if re.search('[a-zA-Z]', complaint) is None:# if there are no letters in the complaint, return empty, will be removed in later processing
        return ""
    complaint = ' '.join([word for word in complaint.split() if word not in cachedStopWords]) # remove stopwords at end after preprocessing (alex) 
    return complaint

def getComplaint(row):
    complaint2 = row.get("COMPLAINT_2")
    if not pd.isnull(complaint2):
        if "[INSPECTION LOG #:" in complaint2: # Remove inspection log section from C2
            complaintStrippedList = complaint2.split("]")[1:]
            complaintFinal = "]".join(complaintStrippedList)
        else:
            complaintFinal = complaint2
        if row.get("CITY")=="US_CHICAGO": # if Chicago, concatenate the two
            complaintFinal = row.get("COMPLAINT_1") + " "+ complaintFinal
        complaintProcessed = preProcess(complaintFinal)
        if complaintProcessed == "" or re.search('[a-zA-Z]', complaintProcessed) is None: # if nothing or no letters
            return preProcess(row.get("COMPLAINT_1"))
        return complaintProcessed
    complaintProcessed = preProcess(row.get("COMPLAINT_1"))
    return complaintProcessed

results = dataFiltered.apply(lambda row: getComplaint (row),axis=1)
print(results[results.isna()].shape)

dataFiltered["complaint"] = results
print(dataFiltered[dataFiltered.complaint.isna()].shape)

dataFiltered["CATEGORY_SUB"] = dataFiltered["CATEGORY_SUB"].str.strip()

(0,)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(0, 11)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
dataFiltered.complaint.head(5)

0    questionable business practices regarding oils...
1    constructiondemolition debris dumped back yard...
2    replacing windows sandblasting causing dust ar...
4    property causing hazardous conditions public w...
5    construction work causing dust investigation d...
Name: complaint, dtype: object

# Start looking at data

** remove any rows with complaints shorter than 2 characters **


In [17]:
mask = (dataFiltered["complaint"].str.len() >=2)
dataFiltered = dataFiltered.loc[mask]
print(dataFiltered.shape)

(1156906, 11)


** Everything that has occured 5 times or more**

In [ ]:
df5plus=pd.DataFrame(dataFiltered["complaint"].value_counts()[dataFiltered["complaint"].value_counts()>=5])

In [20]:
display(df5plus.tail(20))

,complaint
ftc trash entire street per caller,5
needs recycling bin replaced,5
festival del taco,5
transfer amdcottrellrequesting gal recycling cart request entered web refer intake questions description,5
caller trash missed thursday limbs gave id,5
gang tagging,5
needs recycling bins removed,5
pile limbs missed last friday picked yet today,5
toyota camry,5
bags must sticker attached n,5


### Investigate potential autmated entries

In [28]:
# good list
#df5plus[df5plus.index.map(lambda x: any(substring in x for substring in ['system','online','report']))]
# cdph
df5plus[df5plus.index.map(lambda x: any(substring in x for substring in ['information may available','dataset']))]

,complaint
residential area information may available cdph environmental inspections dataset,64
junkyard issues information may available cdph environmental inspections dataset,16
failure pay operation permit information may available cdph environmental inspections dataset,13
check violations information may available cdph environmental inspections dataset,11
burning debris gal drum information may available cdph environmental inspections dataset,9
failure pay certificate operation permit information may available cdph environmental inspections dataset,8
strong odors area information may available cdph environmental inspections dataset,8
residential area noise ac unit information may available cdph environmental inspections dataset,7
ca court date open burning information may available cdph environmental inspections dataset,7
residential area site assessment information may available cdph environmental inspections dataset,6


## See unique complaints of length 2, 3, and 4

In [29]:
for i in [2,3,4]:
  display(dataFiltered['complaint'][dataFiltered["complaint"].str.len() ==i].unique())

array(['bu', 'mh', 'lp', 'tv', 'sl', 'sw', 'dw', 'ps', 'bo', 'co', 'cv',
       'mo', 'hi', 'bc', 'cd', 'bz', 'bl', 'ct', 'bp', 'ts', 'ta', 'na',
       'rv', 'ms', 'te', 'ba', 'ft', 'cc', 'aa', 'md', 'ru', 'kv', 'kn',
       'ac', 'sa', 'en', 'sr', 'eg', 'os', 'ow', 'cw', 'cs', 'sb', 'sc',
       'cb', 'wb', 'pb', 'pc', 'cp', 'sp', 'bs', 'pw', 'mt', 'mr', 'yo',
       'wp', 'rm', 'tm', 'ym', 'ns', 'wc', 'gs', 'nl', 'ss', 'tr', 'ot',
       'ro', 'dd', 'xx', 'rb', 'tb', 'kh', 'jm', 'er', 'js', 'ja', 'go',
       'th', 'qq', 'dk', 'st', 'ew', 'lx', 'af'], dtype=object)

array(['bup', 'gal', 'psf', 'two', 'owl', 'mhs', 'tvd', 'wwc', 'lot',
       'ttv', 'scl', 'act', 'big', 'top', 'lid', 'hey', 'nov', 'rtc',
       'yes', 'ftc', 'van', 'red', 'apt', 'set', 'hea', 'mat', 'tel',
       'msd', 'che', 'wic', 'dre', 'rms', 'suv', 'elm', 'lso', 'cou',
       'air', 'sos', 'mfd', 'bar', 'sfd', 'rsp', 'mob', 'per', 'bin',
       'bpp', 'ohc', 'bld', 'bbs', 'foe', 'udf', 'est', 'one', 'ftv',
       'frt', 'fls', 'apa', 'unk', 'bbq', 'mil', 'cat', 'see', 'bis',
       'bag', 'icy', 'mrc', 'ref', 'att', 'tir', 'mos', 'pot', 'tvs',
       'h h', 'tub', 'acs', 'spa', 'cor', 'dog', 'fix', 'new', 'pos',
       'fyc', 'hwy', 'fyi', 'tvv', 'rat', 'rug', 'add', 'bed', 'nod',
       'dan', 'fan', 'hud', 'fit', 'tvx', 'may', 'vac', 'car', 'sfe',
       'rim', 'sec', 'cah', 'tql', 'lbs', 'cab', 'csb', 'pbs', 'scp',
       'pcs', 'psc', 'pgs', 'pcg', 'pcb', 'psb', 'scw', 'trm', 'rmf',
       'ags', 'use', 'c w', 'scb', 'bpg', 'spc', 'p b', 'p c', 'c b',
       'p g', 'spb',

array(['acct', 'tree', 'apts', 'back', 'file', 'cave', 'gart', 'rear',
       'pick', 'dotd', 'none', 'aact', 'lake', 'help', 'cart', 'ofmh',
       'wbrz', 'need', 'test', 'odor', 'home', 'mhof', 'tear', 'mhmh',
       'ofdw', 'time', 'ofps', 'cvin', 'wash', 'bldg', 'sign', 'flow',
       'area', 'pole', 'lamp', 'dear', 'rats', 'heat', 'mice', 'pous',
       'mold', 'rock', 'dish', 'bugs', 'tire', 'unit', 'couh', 'fire',
       'bath', 'sink', 'love', 'wich', 'pipe', 'zone', 'walk', 'surf',
       'reno', 'tall', 'prop', 'rugs', 'curb', 'crew', 'nepw', 'muse',
       'bags', 'icey', 'teak', 'kvbv', 'yoga', 'ital', 'vbml', 'vine',
       'quad', 'ford', 'mixx', 'cars', 'hill', 'desk', 'door', 'wall',
       'says', 'matt', 'dssg', 'boat', 'elis', 'neff', 'gabp', 'btwn',
       'zbgb', 'swap', 'melt', 'deer', 'cell', 'bank', 'wcet', 'dogs',
       'fttc', 'yard', 'ants', 'zang', 'leak', 'room', 'side', 'feet',
       'sofa', 'jeep', 'road', 'safe', 'cans', 'junk', 'duel', 'ites',
      

In [30]:
for word in ['oil','dog','cat','rat']:
  print(word,":")
  display(dataFiltered[dataFiltered["complaint"]==word].groupby(["CATEGORY_MAIN","CATEGORY_SUB"]).count())
  
for word in ['mold','mice','rats','sofa']:
  print(word,":")
  display(dataFiltered[dataFiltered["complaint"]==word].groupby(["CATEGORY_MAIN","CATEGORY_SUB"]).count())

oil :


,,index,COMPLAINT_ID,CITY,COMPLAINT DATE,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,complaint
CATEGORY_MAIN,CATEGORY_SUB,,,,,,,,,
environment,environment_dumping,4,4,4,4,0,4,4,4,4


dog :


,,index,COMPLAINT_ID,CITY,COMPLAINT DATE,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,complaint
CATEGORY_MAIN,CATEGORY_SUB,,,,,,,,,
environment,environment_dead_animal,5,5,5,5,5,5,5,0,5


cat :


...

index  COMPLAINT_ID  CITY  COMPLAINT DATE  \
CATEGORY_MAIN CATEGORY_SUB                                                      
housing       housing_general          17            17    17              17   
              housing_mold              2             2     2               2   
              housing_safety            1             1     1               1   
public_health publichealth_general      2             2     2               2   
              publichealth_pests      494           494   494             494   

                                    DEPT_311  CODE_311  COMPLAINT_1  \
CATEGORY_MAIN CATEGORY_SUB                                            
housing       housing_general              1        17           17   
              housing_mold                 2         2            2   
              housing_safety               1         1            1   
public_health publichealth_general         2         2            2   
              publichealth_pests         494       494          494   

                                    COMPLAINT_2  complaint  
CATEGORY_MAIN CATEGORY_SUB                                  
housing       housing_general                16         17  
              housing_mold                    0          2  
              housing_safety                  0          1  
public_health publichealth_general            0          2  
              publichealth_pests              0        494

rats :


index  COMPLAINT_ID  CITY  \
CATEGORY_MAIN CATEGORY_SUB                                        
environment   environment_dumping         1             1     1   
              environment_general         1             1     1   
              environment_overgrowth      1             1     1   
governance    governance_general          1             1     1   
housing       housing_general             1             1     1   
public_health publichealth_general        3             3     3   
              publichealth_pests        210           210   210   

                                      COMPLAINT DATE  DEPT_311  CODE_311  \
CATEGORY_MAIN CATEGORY_SUB                                                 
environment   environment_dumping                  1         0         1   
              environment_general                  1         0         1   
              environment_overgrowth               1         0         1   
governance    governance_general                   1         0         1   
housing       housing_general                      1         1         1   
public_health publichealth_general                 3         3         3   
              publichealth_pests                 210       210       210   

                                      COMPLAINT_1  COMPLAINT_2  complaint  
CATEGORY_MAIN CATEGORY_SUB                                                 
environment   environment_dumping               1            1          1  
              environment_general               1            1          1  
              environment_overgrowth            1            1          1  
governance    governance_general                1            0          1  
housing       housing_general                   1            0          1  
public_health publichealth_general              3            0          3  
              publichealth_pests              210            0        210

sofa :


index  COMPLAINT_ID  CITY  \
CATEGORY_MAIN  CATEGORY_SUB                                                
environment    environment_dumping                10            10    10   
               environment_garbage_collection    177           177   177   
               environment_litter                  1             1     1   
infrastructure infrastructure_water                1             1     1   

                                               COMPLAINT DATE  DEPT_311  \
CATEGORY_MAIN  CATEGORY_SUB                                               
environment    environment_dumping                         10         0   
               environment_garbage_collection             177       177   
               environment_litter                           1         0   
infrastructure infrastructure_water                         1         0   

                                               CODE_311  COMPLAINT_1  \
CATEGORY_MAIN  CATEGORY_SUB                                            
environment    environment_dumping                   10           10   
               environment_garbage_collection       177          177   
               environment_litter                     1            1   
infrastructure infrastructure_water                   1            1   

                                               COMPLAINT_2  complaint  
CATEGORY_MAIN  CATEGORY_SUB                                            
environment    environment_dumping                      10         10  
               environment_garbage_collection            0        177  
               environment_litter                        1          1  
infrastructure infrastructure_water                      1          1

In [31]:
dataFiltered[dataFiltered["complaint"].str.len() ==4].groupby('complaint').count().reset_index().sort_values('index', ascending=False).head(10)

,complaint,index,COMPLAINT_ID,CITY,COMPLAINT DATE,DEPT_311,CODE_311,CATEGORY_MAIN,CATEGORY_SUB,COMPLAINT_1,COMPLAINT_2
3,acct,1428,1428,1428,1428,1428,1428,1428,1428,1428,0
251,tags,943,943,943,943,927,943,943,943,943,16
263,tire,734,734,734,734,727,734,734,734,734,6
167,mold,705,705,705,705,700,705,705,705,705,5
120,heat,662,662,662,662,656,662,662,662,662,6
162,mice,516,516,516,516,500,516,516,516,516,16
235,sign,227,227,227,227,6,227,227,227,227,185
213,rats,218,218,218,218,214,218,218,218,218,3
203,pole,190,190,190,190,12,190,190,190,190,182
243,sofa,189,189,189,189,177,189,189,189,189,12


# Second Filter

In [53]:
# eliminate stop_phrases
exclusions = '|'.join(stop_phrases)  #first_word = re.sub(exclusions, '', first_word)
dataFiltered['complaint']=dataFiltered['complaint'].map(lambda x: re.sub(exclusions,'',x))

# keep only len 4 and greater
dataFiltered = dataFiltered.loc[(dataFiltered["complaint"].str.len() >=4)]
print(dataFiltered.shape)

(965251, 11)


In [ ]:
df5plus_v2=pd.DataFrame(dataFiltered["complaint"].value_counts()[dataFiltered["complaint"].value_counts()>=5])

In [55]:
df5plus_v2.head(40)

,complaint
graffiti removal,17243
pothole street,16791
street light,12615
rodent baiting rat complaint,10138
abandoned vehicle,8176
alley light,8165
tree debris,6766
building violation,5602
locked,4404
sanitation code violation,4294


In [56]:
display(dataFiltered[dataFiltered["complaint"]=='locked'].groupby(["CATEGORY_MAIN","CATEGORY_SUB"]).count())
display(dataFiltered[dataFiltered["complaint"]=='reported'].groupby(["CATEGORY_MAIN","CATEGORY_SUB"]).count())  # shouldn't be here. what is wrong?

,,index,COMPLAINT_ID,CITY,COMPLAINT DATE,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,complaint
CATEGORY_MAIN,CATEGORY_SUB,,,,,,,,,
planning,planning_unsafe_environment,4404,4404,4404,4404,0,4404,4404,4404,4404


index  COMPLAINT_ID  CITY  \
CATEGORY_MAIN CATEGORY_SUB                                           
governance    governance_general             1             1     1   
public_health publichealth_animal_feces      1             1     1   
street        street_cleaning              151           151   151   
              street_repair                  1             1     1   
vandalism     vandalism_graffiti             4             4     4   

                                         COMPLAINT DATE  DEPT_311  CODE_311  \
CATEGORY_MAIN CATEGORY_SUB                                                    
governance    governance_general                      1         0         1   
public_health publichealth_animal_feces               1         0         1   
street        street_cleaning                       151         0       151   
              street_repair                           1         0         1   
vandalism     vandalism_graffiti                      4         0         4   

                                         COMPLAINT_1  COMPLAINT_2  complaint  
CATEGORY_MAIN CATEGORY_SUB                                                    
governance    governance_general                   1            1          1  
public_health publichealth_animal_feces            1            1          1  
street        street_cleaning                    151          151        151  
              street_repair                        1            1          1  
vandalism     vandalism_graffiti                   4            4          4

In [57]:
#df5plus_v2
df5plus_v2[df5plus_v2.index.map(lambda x: any(substring in x for substring in ['system','online','report','information may available','dataset','batch']))]

,complaint
batch group,2181
reported,158
need vehicle tow report,118
resident originally called put request new resident recycling cart due change contractors updating system new request per nelle fruge sanitation manager,113
report litter hotspot,79
reported ps,28
set clock system time,26
resident originally called put request damaged lid cart due change contractors updating system new request per nelle fruge sanitation manager,24
graffiti reported weeks ago thank,12
resident originally called put request damaged lid cart updating system new request per nelle fruge sanitation manager,10


## Remove stop complaints

In [ ]:
stop_complaints=['reported',\
                'reported ps',\
                #'locked',\  # one word locked doesn't say much, but all related to unsafe planning, so dont want to miss locked [door, fire escape, etc] with pic
                'batch group',\
                'mt airy nep']
                 


In [ ]:
dummy=dataFiltered[~dataFiltered['complaint'].isin(stop_complaints)]

In [ ]:
df5plus_v3=pd.DataFrame(dummy["complaint"].value_counts()[dummy["complaint"].value_counts()>=5])

In [64]:
df5plus_v3[df5plus_v3.index.map(lambda x: any(substring in x for substring in ['system','online','report','information may available','dataset','batch']))]

,complaint
need vehicle tow report,118
resident originally called put request new resident recycling cart due change contractors updating system new request per nelle fruge sanitation manager,113
report litter hotspot,79
set clock system time,26
resident originally called put request damaged lid cart due change contractors updating system new request per nelle fruge sanitation manager,24
graffiti reported weeks ago thank,12
park hub gang activity vcv increased tagging field due another rival tagging crew ws fcb green sox baseball club reported pd additionally painting graffiti overwhelmed,10
resident originally called put request damaged lid cart updating system new request per nelle fruge sanitation manager,10
crews didnt leave notice jeff reported trash without stickers,9
graffiti underneath bay farm island bridge photo taken evidentiary purposes gone online caltrans website created service request right,7


In [65]:
dummy.shape

(961399, 11)

In [67]:
df5plus_v3.head(100)

,complaint
graffiti removal,17243
pothole street,16791
street light,12615
rodent baiting rat complaint,10138
abandoned vehicle,8176
alley light,8165
tree debris,6766
building violation,5602
locked,4404
sanitation code violation,4294


In [76]:
for i in [10,50,100,500,1000]:
  print('top',i,'complaints make up',100*round(df5plus_v3.head(i)['complaint'].sum() / dummy.shape[0],4),'% of total complaints')
  #print(i)

top 10 complaints make up 9.8 % of total complaints
top 50 complaints make up 16.88 % of total complaints
top 100 complaints make up 20.580000000000002 % of total complaints
top 500 complaints make up 28.470000000000002 % of total complaints
top 1000 complaints make up 31.71 % of total complaints


In [ ]:
# 'word nep'  in re 

# Save data

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html


In [ ]:
dummy.to_csv('combined_trainingdata_20181108.tsv',sep='\t')

In [82]:
from google.colab import files
files.download('combined_trainingdata_20181108.tsv')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 59358, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [80]:
# 2. Create & upload a file text file.
uploaded = drive.CreateFile({'title': 'combined_trainingdata_20181108.tsv'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))


Uploaded file with ID 1CDKVW8JeH--GWaZLCK2nKB3XX8HikkQw


In [ ]:
# 3. Load a file by ID and print its contents.
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [ ]:
# Save data
